In [205]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle


import sys, os
sys.path.append(os.path.abspath('..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

plt.rcParams['lines.linewidth'] = 0.5 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
pair = 'BTC_USDT'
timeframe = '1d'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

(2114, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2017-08-17,1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,1503187200000,4120.98,4211.08,4032.62,4086.29,467.083022


In [144]:
wdf = odf.iloc[-30 * 6:,:]
wdf

,timestamp,open,high,low,close,volume
dtime,,,,,,
2022-12-03,1670025600000,17092.13,17188.98,16858.74,16885.20,154542.57306
2022-12-04,1670112000000,16885.20,17202.84,16878.25,17105.70,178619.13387
2022-12-05,1670198400000,17106.65,17424.25,16867.00,16966.35,233703.29225
2022-12-06,1670284800000,16966.35,17107.01,16906.37,17088.96,218730.76883
2022-12-07,1670371200000,17088.96,17142.21,16678.83,16836.64,220657.41334
...,...,...,...,...,...,...
2023-05-27,1685145600000,26705.93,26895.00,26551.00,26854.27,15095.68670
2023-05-28,1685232000000,26854.28,28261.32,26764.36,28065.00,43916.00855
2023-05-29,1685318400000,28065.01,28447.14,27524.60,27736.40,42385.41945


In [213]:
def plot_candles(wdf, ax):
    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .4, .1
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)   


In [237]:
fibs = np.array([0, 0.236, 0.382, 0.5, 0.618, 0.786, 1])
# dfibs = pd.Series(fibs, index=wdf.index)

@interact(x0=(0,wdf.shape[0], 1),x1=(0,wdf.shape[0], 1),x2=(0,wdf.shape[0], 1),xx=(0,wdf.shape[0], 1))
def printer(x0=31,x1=57,x2=97,xx=wdf.shape[0] // 2):
    wlen = wdf.shape[0]
    x_last = wlen - 1
    fig = plt.figure(figsize=(20,9))
    ax, ax1 = fig.subplots(2,1, sharex=True, height_ratios=[2,1])
    y = wdf.close.values
    idx0, idx1, idx2, idxx = wdf.iloc[[x0,x1,x2, xx]].index.values
    y0, y1, y2 = wdf.close.iloc[[x0,x1,x2]]

    # y_d = y2 - y0
    k = (y2 - y0) / (x2 - x0)
    
    y_lower = (pd.Series(range(wdf.shape[0])) - x1) * k + y1
    y_upper = (pd.Series(range(wdf.shape[0])) - x0) * k + y0


    y_tran = y - y_upper
    y_tran.index = wdf.index
    yt0, yt1, yt2 = y_tran.iloc[[x0,x1,x2]]

    y_fibs =  [y_lower + (y_upper - y_lower) * fib for fib in fibs] +  [y_lower + (y_upper - y_lower) * - fib for fib in fibs] +  [y_lower + (y_upper - y_lower) * (1 + fib) for fib in fibs]

    # wdf.close.plot(ax=ax,lw=0.5)
    plot_candles(wdf,ax)
    # ax.set_yscale('symlog')

    # y_d = 
    plt.xticks(rotation=30, ha='right')
    ax.axvline(idx0, c='r'); ax.axvline(idx1, c='g'); ax.axvline(idx2, c='b');
    ax1.axvline(idx0, c='r'); ax1.axvline(idx1, c='g'); ax1.axvline(idx2, c='b');
    ax.axvline(idxx, linestyle='dotted');ax1.axvline(idxx, linestyle='dotted')
    ax.axhline(y0, linestyle='dashed');ax.axhline(y1, linestyle='dashed');ax.axhline(y2, linestyle='dashed');
    
    ax.plot(wdf.index[[0,-1]].values,y_upper.iloc[[0,-1]].values, c='maroon')
    ax.plot(wdf.index[[0,-1]].values,y_lower.iloc[[0,-1]].values, c='maroon')
    # ax1.axhline(0)
    axtw = ax1.twinx()
    # y_tran.plot(ax=axtw, c='g')
    # axtw.axhline(y_tran[x0]);axtw.axhline(y_tran[x1])
    y_norm = pd.Series( y_tran / yt1, index=wdf.index)

    y_norm.plot(ax=axtw, c='g')
    # y_norm = pd.Series(np.sin( y / y_tran[x1]), index=wdf.index) 
    # y_norm_p = np.cos( - pi / 2 + y_norm * pi)  #/ 2 + 0.5
    y_norm_p =  (y_norm  % 1)
    y_norm_p.plot(ax=ax1, c='r')
    # ax1.grid(); ax.grid()
    # dfibs.plot(ax=ax1)
    for fib in y_fibs:
        ax.plot(wdf.index[[0,-1]].values,fib.iloc[[0,-1]].values, c='maroon')
    for fib in fibs:
        ax1.axhline(fib)
        # ax1.axhline(-fib)


interactive(children=(IntSlider(value=31, description='x0', max=180), IntSlider(value=57, description='x1', ma…

In [112]:
fibs = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

In [178]:
y = wdf.close.values
xx=0
x0,x1,x2 = 20, 30, 50
# idx0, idx1, idx2, idxx = wdf.iloc[[x0,x1,x2, xx]].index.values
y0, y1, y2 = wdf.close.iloc[[x0,x1,x2]]

# y_d = y2 - y0
k = (y2 - y0) / (x2 - x0)

y_upper = ((pd.Series(range(wdf.shape[0])) - x0) * k + y0)
y_upper.iloc[[0,-1]].values

array([12825.58 , 48204.214])

In [185]:

xx=0
x0,x1,x2 = 20, 30, 50
# fig = plt.figure(figsize=(20,6))
# ax, ax1 = fig.subplots(2,1, sharex=True)
y = wdf.close.values
idx0, idx1, idx2 = wdf.iloc[[x0,x1,x2]].index.values
y0, y1, y2 = wdf.close.iloc[[x0,x1,x2]]

# y_d = y2 - y0
k = (y2 - y0) / (x2 - x0)

y_tran = y - ((pd.Series(range(wdf.shape[0])) - x0) * k + y0)
y_tran.index = wdf.index
yt0, yt1, yt2 = y_tran.iloc[[x0,x1,x2]]

y_lower = (pd.Series(range(wdf.shape[0])) - x1) * k + y1
y_upper = (pd.Series(range(wdf.shape[0])) - x0) * k + y0


y_tran = y - y_upper
y_tran.index = wdf.index
yt0, yt1, yt2 = y_tran.iloc[[x0,x1,x2]]

y_fibs = np.multiply(fibs, y_lower + (y_upper - y_lower))
y_fibs

ValueError: operands could not be broadcast together with shapes (7,) (180,) 